In [1]:
from google.colab import userdata, drive
import os
drive.mount('/content/drive')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')

Mounted at /content/drive


In [2]:
!pip install kaggle
!kaggle datasets download -d kacky355/belka-train-valid-tfrecords-1d-preprocessed
!unzip belka-train-valid-tfrecords-1d-preprocessed.zip


Dataset URL: https://www.kaggle.com/datasets/kacky355/belka-train-valid-tfrecords-1d-preprocessed
License(s): unknown
100% 1.98G/1.98G [00:28<00:00, 90.4MB/s]
100% 1.98G/1.98G [00:28<00:00, 75.4MB/s]
Archive:  belka-train-valid-tfrecords-1d-preprocessed.zip
  inflating: logs/main.log           
  inflating: tf_idx/train_00.idx     
  inflating: tf_idx/train_01.idx     
  inflating: tf_idx/train_02.idx     
  inflating: tf_idx/train_03.idx     
  inflating: tf_idx/train_04.idx     
  inflating: tf_idx/train_05.idx     
  inflating: tf_idx/train_06.idx     
  inflating: tf_idx/train_07.idx     
  inflating: tf_idx/train_08.idx     
  inflating: tf_idx/train_09.idx     
  inflating: tf_idx/train_10.idx     
  inflating: tf_idx/train_11.idx     
  inflating: tf_idx/train_12.idx     
  inflating: tf_idx/train_13.idx     
  inflating: tf_idx/train_14.idx     
  inflating: tf_idx/train_15.idx     
  inflating: tf_idx/train_16.idx     
  inflating: tf_idx/train_17.idx     
  inflating: tf_idx/

In [15]:

!pip install rdkit
!pip install lightning
!pip install polars

!pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-cuda120
!pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-tf-plugin-cuda120

!pip install git+https://github.com/kacky355/my_libraries.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 23.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x8

In [3]:
%%writefile config.py
import os
import glob
import torch

class CFG:

    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    DEVICE_ID = 0 if DEVICE=='cuda' else None

    BATCH_SIZE= 4096
    NUM_TRAINS = 91_854_569
    NUM_VALIDS = 6_561_041
    STEPS_PER_EPOCH_TRAIN = (NUM_TRAINS -1) //BATCH_SIZE +1
    STEPS_PER_EPOCH_VALID = (NUM_VALIDS -1) //BATCH_SIZE +1

    BASE_DIR = '/content/drive/MyDrive/BELKA_model/kaggle/working'
    DATA_SOURCE = '/content'
    TRAINS = glob.glob(os.path.join(DATA_SOURCE, 'train/*'))
    TRAINS.sort()
    TRAIN_IDX = glob.glob(os.path.join(DATA_SOURCE, 'tf_idx', 'train_*.idx'))
    TRAIN_IDX.sort()
    VALIDS = glob.glob(os.path.join(DATA_SOURCE, 'valid/*'))
    VALIDS.sort()
    v_id = glob.glob(os.path.join(DATA_SOURCE, 'tf_idx', 'valid_*.idx'))
    VALID_IDX = v_id
    VALID_IDX.sort()

    SEED = 2024


    FEATURES = [f'enc{i}' for i in range(142)]
    TARGETS = ['bind1', 'bind2', 'bind3']
    COLUMNS = FEATURES + TARGETS

    NUM_CLASSES = 3
    SEQ_LENGTH = 142

    LR = 1e-3
    WD = 0.05




Writing config.py


In [4]:
# from config import CFG
# import glob
# import os
# glob.glob(os.path.join(CFG.DATA_SOURCE, 'valid/*'))
# CFG.VALID_IDX

In [5]:
# import os
# import glob
# from config import CFG


# from nvidia.dali import pipeline_def, Pipeline
# import nvidia.dali.fn as fn
# import nvidia.dali.types as types
# import nvidia.dali.tfrecord as tfrec
# from nvidia.dali.plugin.pytorch import DALIGenericIterator, LastBatchPolicy
# from nvidia.dali.plugin.tf import DALIDataset

# v_id = glob.glob(os.path.join(CFG.DATA_SOURCE, 'tf_idx', 'valid_*.idx'))
# VALID_IDX = v_id
# VALID_IDX.sort()

# @pipeline_def
# def belka_pipeline(device, paths, idxs, seed,shard_id=0, num_shards=1, is_train=True):
#     device_id = Pipeline.current().device_id

#     inputs = fn.readers.tfrecord(
#         path = paths,
#         index_path = idxs,
#         features={
#             "x": tfrec.FixedLenFeature([CFG.SEQ_LENGTH], tfrec.int64, 0),
#             "y": tfrec.FixedLenFeature([CFG.NUM_CLASSES], tfrec.float32, .0)
#         },
#         random_shuffle=is_train,
#         num_shards=num_shards,
#         shard_id=shard_id,
#         initial_fill=CFG.BATCH_SIZE,
#         seed=seed,
#         name='Reader'
#     )
#     x = inputs['x']
#     y = inputs['y']
#     if device=='cuda':
#         x = x.gpu()
#         y = y.gpu()
#     return x,y


In [6]:
# from config import CFG

# import tensorflow as tf


# torch_train_pipe = belka_pipeline(
#     batch_size=CFG.BATCH_SIZE,
#     num_threads=4,
#     device_id=CFG.DEVICE_ID,
#     device=CFG.DEVICE,
#     paths=CFG.TRAINS,
#     idxs=CFG.TRAIN_IDX,
#     seed=CFG.SEED + 1
# )

# tf_train_pipe = belka_pipeline(
#     batch_size=CFG.BATCH_SIZE,
#     num_threads=4,
#     device_id=CFG.DEVICE_ID,
#     device=CFG.DEVICE,
#     paths=CFG.TRAINS,
#     idxs=CFG.TRAIN_IDX,
#     seed=CFG.SEED + 1
# )

# torch_valid_pipe = belka_pipeline(
#     batch_size=CFG.BATCH_SIZE,
#     num_threads=4,
#     device_id=CFG.DEVICE_ID,
#     device=CFG.DEVICE,
#     paths=CFG.VALIDS,
#     idxs=VALID_IDX,
#     seed=CFG.SEED-2
# )
# tf_valid_pipe = belka_pipeline(
#     batch_size=CFG.BATCH_SIZE,
#     num_threads=4,
#     device_id=CFG.DEVICE_ID,
#     device=CFG.DEVICE,
#     paths=CFG.VALIDS,
#     idxs=VALID_IDX,
#     seed=CFG.SEED-4
# )

# class LightningWrapper(DALIGenericIterator):
#     def __init__(self, *kargs, **kwargs):
#         super().__init__(*kargs, **kwargs)
#     def __next__(self):
#         out = super().__next__()
#         out = out[0]
#         return [out[k] for k in self.output_map]


# valid_loader = LightningWrapper(torch_valid_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.PARTIAL)
# train_loader = LightningWrapper(torch_train_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.DROP)


# tf_train = DALIDataset(pipeline=tf_train_pipe,
#                        output_dtypes=(tf.int64, tf.float32),
#                        output_shapes=((CFG.BATCH_SIZE, 142), (CFG.BATCH_SIZE, 3)),
#                        batch_size=CFG.BATCH_SIZE,
#                        device_id=None)
# tf_valid = DALIDataset(pipeline=tf_valid_pipe,
#                        output_dtypes=(tf.int64, tf.float32),
#                        output_shapes=((CFG.BATCH_SIZE, 142), (CFG.BATCH_SIZE, 3)),
#                        batch_size=CFG.BATCH_SIZE,
#                        device_id=None)

In [7]:
from config import CFG
CFG.VALIDS

['/content/valid/BELKA_valid.tfrecords.0',
 '/content/valid/BELKA_valid.tfrecords.1',
 '/content/valid/BELKA_valid.tfrecords.2',
 '/content/valid/BELKA_valid.tfrecords.3',
 '/content/valid/BELKA_valid.tfrecords.4',
 '/content/valid/BELKA_valid.tfrecords.5',
 '/content/valid/BELKA_valid.tfrecords.6',
 '/content/valid/BELKA_valid.tfrecords.7']

In [8]:
import tensorflow as tf
from config import CFG

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def tfmap(example):
    features = tf.io.parse_single_example(
        example,
        features={
            "x": tf.io.FixedLenFeature([CFG.SEQ_LENGTH], tf.int64),
            "y": tf.io.FixedLenFeature([CFG.NUM_CLASSES], tf.float32)
        }
    )
    return features['x'], features['y']


In [9]:
# i = 0
# for x,y in tf.data.TFRecordDataset(CFG.VALIDS).map(tfmap).take(5):
#     if i%4096

In [10]:
val_loader = tf.data.TFRecordDataset(CFG.VALIDS)\
    .map(tfmap)\
    .batch(4096)\
    .prefetch(2)

In [11]:
CFG.STEPS_PER_EPOCH_VALID

1602

In [38]:
from tqdm.notebook import tqdm
i = 0
for x, y in tqdm(val_loader):
    if i%200 ==0:
        print(i,x.shape , y.shape)
    if i == CFG.STEPS_PER_EPOCH_VALID-1:
        print(i,x.shape , y.shape)

        break
    i += 1

0it [00:00, ?it/s]

0 (4096, 142) (4096, 3)
200 (4096, 142) (4096, 3)
400 (4096, 142) (4096, 3)
600 (4096, 142) (4096, 3)
800 (4096, 142) (4096, 3)
1000 (4096, 142) (4096, 3)
1200 (4096, 142) (4096, 3)
1400 (4096, 142) (4096, 3)
1600 (4096, 142) (4096, 3)
1601 (3345, 142) (3345, 3)


In [9]:
%%writefile res_model.py
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers as L

class ModelCFG:
    LR = 1e-3
    WD = 0.05



def bottleneck_residual(x, input_dim, output_dim, strides=1):
    params={
        'padding' : 'same',
        'kernel_initializer' : 'he_normal',
        'use_bias' : True
    }

    hidden_dim = output_dim // 4
    h1 = L.Conv1D(filters=hidden_dim, kernel_size=1, strides=strides, **params)(x)
    h1 = L.BatchNormalization()(h1)
    h1 = L.ReLU()(h1)
    h1 = L.Conv1D(filters=hidden_dim, kernel_size=3, strides=1, **params)(h1)
    h1 = L.BatchNormalization()(h1)
    h1 = L.ReLU()(h1)
    h1 = L.Conv1D(filters=output_dim, kernel_size=1, strides=1, **params)(h1)
    h1 = L.BatchNormalization()(h1)

    if input_dim != output_dim:
        h2 = L.Conv1D(filters=output_dim, kernel_size=1, strides=strides, **params)(x)
        h2 = L.BatchNormalization()(h2)
    else:
        h2 = x

    h = L.Add()([h1, h2])
    h = L.ReLU()(h)

    return h

def my_model(strategy):
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 128
        hidden_dim = 128

        inputs = L.Input(shape=(INP_LEN,), dtype='int32')
        x = L.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)

        x = bottleneck_residual(x, hidden_dim, NUM_FILTERS)
        x = bottleneck_residual(x, NUM_FILTERS, NUM_FILTERS)
        x = bottleneck_residual(x, NUM_FILTERS, NUM_FILTERS)

        x = bottleneck_residual(x, NUM_FILTERS, NUM_FILTERS*2)
        x = bottleneck_residual(x, NUM_FILTERS*2, NUM_FILTERS*2)
        x = bottleneck_residual(x, NUM_FILTERS*2, NUM_FILTERS*2)

        x = bottleneck_residual(x, NUM_FILTERS*2, NUM_FILTERS*3, 2)
        x = bottleneck_residual(x, NUM_FILTERS*3, NUM_FILTERS*3)
        x = bottleneck_residual(x, NUM_FILTERS*3, NUM_FILTERS*3)
        x = bottleneck_residual(x, NUM_FILTERS*3, NUM_FILTERS*3)

        x = L.GlobalMaxPooling1D()(x)
        x = L.Dense(1024, activation='relu')(x)
        x = L.Dropout(0.1)(x)
        x = L.Dense(1024, activation='relu')(x)
        x = L.Dropout(0.1)(x)
        x = L.Dense(512, activation='relu')(x)
        x = L.Dropout(0.1)(x)

        outputs = L.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=ModelCFG.LR, weight_decay = ModelCFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

Overwriting res_model.py


In [23]:
from config import CFG
import tensorflow as tf
import torch
from torchmetrics.classification import MultilabelAveragePrecision as mAP

try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
except:
    try:
        assert torch.cuda.is_available()
        strategy = tf.distribute.MirroredStrategy()
        print("Running on GPU")
        print("REPLICAS: ", strategy.num_replicas_in_sync)
    except:
        strategy = tf.distribute.get_strategy()
        print("Running on CPU")


Running on GPU
REPLICAS:  1


In [11]:
# %%writefile SeX_model.py
from config import CFG
from dataclasses import dataclass
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers

@dataclass
class SeX_ModelArgs:
    input_dim:int = 128
    channels: int = 256
    conv_kernel_size: int = 3
    seq_length: int = 142
    dropout_rate: float = 0.2
    num_classes: int = 3
    vocab_size: int = 37
    activation: str = 'silu'
    se_reduction: int = 16

    def __post_init__(self):
        self.num_layers = [2,4,4,2]

@dataclass
class SeXc_v3_ModelArgs:
    input_dim = 128
    channels: int = 256
    conv_kernel_size: int = 3
    seq_length: int = 142
    dropout_rate: float = 0.2
    num_classes: int = 3
    vocab_size: int = 37
    activation: str = 'silu'
    se_reduction: int = 16

    def __post_init__(self):
        self.num_layers = [8,8,2]


class ConvSiluBN(layers.Layer):
    def __init__(self, out_size:int,kernel:int=3, strides:int=1, act:bool=True):
        super().__init__()
        self.act = act
        self.bn= layers.BatchNormalization()
        initializer = keras.initializers.HeNormal(CFG.SEED)
        self.conv = layers.SeparableConv1D(out_size, kernel, strides=strides, padding='same',depthwise_initializer=initializer, pointwise_initializer=initializer)

    def call(self, x):
        if self.act:
            x = keras.activations.relu(x)
        x = self.bn(x)
        x = self.conv(x)
        return x

class XceptionLayer(layers.Layer):
    def __init__(self,  out_size_1:int, out_size_2=None,strides:int=1, act:bool=True, pool:bool=False):
        super().__init__()
        if out_size_2 is None:
            out_size_2 = out_size_1

        self.conv1 = ConvSiluBN(out_size_1, act=act)
        self.conv2 = ConvSiluBN(out_size_2)
        if pool:
            self.fc = layers.MaxPool1D(pool_size=3, strides=strides, padding='same')
        else:
            self.fc = ConvSiluBN(out_size_2, strides=strides)
        self.se = SELayer(out_size_2)

    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc(x)
        x = self.se(x)
        return x


class SELayer(layers.Layer):
    def __init__(self,input_size:int , reduction:int=16):
        super().__init__()
        self.avg_pool = layers.GlobalAveragePooling1D(keepdims=True)
        self.dense1 = layers.Dense(
            input_size // reduction,
            activation='silu',
            use_bias = False
        )
        self.dense2 = layers.Dense(
            input_size,
            activation='sigmoid',
            use_bias = False
        )

    def call(self, x):
        y = self.avg_pool(x)
        y = self.dense1(y)
        y = self.dense2(y)

        return x * y


class ResidualLayer(layers.Layer):
    def __init__(self, input_size:int, out_size:int, strides:int=1, pool:bool=False):
        super().__init__()
        self.norm = layers.LayerNormalization(epsilon=1e-5)
        self.xcp = XceptionLayer(out_size,strides=strides,pool=pool)
        self.in_eq_out = input_size == out_size
        if not self.in_eq_out :
            self.conv = layers.Conv1D(out_size,1, strides=strides, padding='same', kernel_initializer='he_normal')
        self.add = layers.Add()


    def call(self, x):
        res = x
        if not self.in_eq_out:
            res = self.conv(res)


        x = self.norm(x)
        x = self.xcp(x)
        x = self.add([x, res])
        return x

class ResidualBlock(layers.Layer):
    def __init__(self,in_size, out_size, num_layer, strides):
        super().__init__()
        self.layers = [ResidualLayer(in_size,in_size) for _ in range(num_layer-1)]
        self.last_layer = ResidualLayer(in_size, out_size, strides=strides, pool=True)

    def call(self,x):
        for layer in self.layers:
            x = layer(x)
        x = self.last_layer(x)
        return x

def make_tpu_model(model_args: SeX_ModelArgs):
    args = model_args
    with strategy.scope():
        inputs = keras.Input(shape=(args.seq_length,), dtype='int32')
        x = layers.Embedding(input_dim=args.vocab_size, output_dim=args.input_dim, input_length=args.seq_length, mask_zero = True)(inputs)
        x = layers.Conv1D(args.channels, 1,padding='same',activation='relu', kernel_initializer='he_normal')(x)
        x = layers.BatchNormalization()(x)

        for i, layer_size in enumerate(args.num_layers, 1):
            x = ResidualBlock(args.channels*i,args.channels*(i+1), layer_size, 2)(x)
            x = layers.Dropout(0.1)(x)

        x = layers.LayerNormalization(epsilon=1e-5)(x)

        x = layers.GlobalMaxPooling1D()(x)


        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)

        outputs = layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model


def make_SX_v3_model(model_args: SeXc_v3_ModelArgs):
    args = model_args
    with strategy.scope():
        inputs = keras.Input(shape=(args.seq_length,), dtype='int32')
        x = layers.Embedding(input_dim=args.vocab_size, output_dim=args.input_dim, input_length=args.seq_length, mask_zero = True)(inputs)
        x = layers.Conv1D(args.channels, 1,padding='same',activation='relu', kernel_initializer='he_normal')(x)
        x = layers.BatchNormalization()(x)

        for i, layer_size in enumerate(args.num_layers, 1):
            strides = 2
            if i % 2 == 0:
                strides = 1
            x = ResidualBlock(args.channels*i,args.channels*(i+1), layer_size, strides)(x)
            x = layers.Dropout(0.1)(x)

        x = layers.LayerNormalization(epsilon=1e-5)(x)

        x = layers.GlobalMaxPooling1D()(x)

        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(0.1)(x)

        outputs = layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model


In [20]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS

class CFG:

    PREPROCESS = False
    PRETRAINED = True
    EPOCHS = 20
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05
    # Number of folds
    NBR_FOLDS = 15

    # Only the first fold selected
    SELECTED_FOLDS = [0,1,2,3,4]
    EXIST_MODELS = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

    SEED = 2222


#1D-CNN model
def OneDCNN_model(strategy):
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 32
        hidden_dim = 128

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=36, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.Adam(learning_rate=CFG.LR, weight_decay = CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model

In [15]:
from res_model import my_model


SeX_args = SeX_ModelArgs()
SeXc_v3_args = SeXc_v3_ModelArgs()
SeX_model = make_tpu_model(SeX_args)
SeXc_v3_model = make_SX_v3_model(SeXc_v3_args)
res_model = my_model(strategy)


In [18]:
SeX_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 142)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 142, 128)          4736      
                                                                 
 conv1d_18 (Conv1D)          (None, 142, 256)          33024     
                                                                 
 batch_normalization_170 (B  (None, 142, 256)          1024      
 atchNormalization)                                              
                                                                 
 residual_block_14 (Residua  (None, 71, 512)           775936    
 lBlock)                                                         
                                                                 
 dropout_24 (Dropout)        (None, 71, 512)           0   

In [16]:
from logger.mylogger import get_my_logger
import os
import time
import random
import numpy as np
import torch
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def set_logger(name):
    now = time.localtime()
    now = time.strftime("%Y-%m-%d-%H-%M-%S", now)
    log_name = f'{name}-{now}.log'
    logger = get_my_logger(CFG.BASE_DIR, log_name)
    return logger
set_seeds(CFG.SEED)
name='rental_cnn'
logger = set_logger(name)



2024-07-08 09:48:58,866 SeX-2024-07-08-09-48-58.log:27 get_my_logger [INFO]: logger has made. log_dir:/content/drive/MyDrive/BELKA_model/kaggle/working/logs


In [41]:
SeX_model.load_weights('/content/drive/MyDrive/BELKA_model/model-SeXception_v2.h5')
preds = SeX_model.predict(val_loader)

1602/1602 [==============================] - 141s 82ms/step


In [46]:
SeXc_v3_model.load_weights('/content/drive/MyDrive/BELKA_model/model_SeXception_v3_fold0_ep9.h5')
preds = SeXc_v3_model.predict(val_loader)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, ((<tf.Tensor 'cond/Identity:0' shape=(None, 142) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 3) dtype=float32>),), {}]


1602/1602 [==============================] - 210s 123ms/step


In [50]:
res_model.load_weights('/content/drive/MyDrive/BELKA_model/model-0.h5')
preds = res_model.predict(val_loader)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, ((<tf.Tensor 'cond/Identity:0' shape=(None, 142) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 3) dtype=float32>),), {}]


1602/1602 [==============================] - 132s 79ms/step


In [25]:
rental_cnn = OneDCNN_model(strategy)
rental_cnn.load_weights('/content/drive/MyDrive/BELKA_model/model-1dcnn_rental.h5')
preds = rental_cnn.predict(val_loader)

1602/1602 [==============================] - 190s 117ms/step


In [26]:
import pandas as pd
val_results = pd.DataFrame({f'bind{i}': preds[:, i] for i in range(3)})


In [27]:
val_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6561041 entries, 0 to 6561040
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   bind0   float32
 1   bind1   float32
 2   bind2   float32
dtypes: float32(3)
memory usage: 75.1 MB


In [28]:
name='rental_cnn'
val_results.to_csv(os.path.join(CFG.BASE_DIR, f'val_results_{name}.csv'))
logger.info(f'val_results write complite!\nfile_name: val_results_{name}.csv')

2024-07-08 09:57:14,289 SeX-2024-07-08-09-48-58.log:3 <cell line: 3> [INFO]: val_results write complite!
file_name: val_results_rental_cnn.csv


In [33]:
test = pd.read_parquet(f'/content/drive/MyDrive/BELKA_model/test_enc.parquet')
test.head()

,enc0,enc1,enc2,enc3,enc4,enc5,enc6,enc7,enc8,enc9,...,enc132,enc133,enc134,enc135,enc136,enc137,enc138,enc139,enc140,enc141
0,8,22,8,8,8,8,29,8,3,5,...,0,0,0,0,0,0,0,0,0,0
1,8,22,8,8,8,8,29,8,3,5,...,0,0,0,0,0,0,0,0,0,0
2,8,22,8,8,8,8,29,8,3,5,...,0,0,0,0,0,0,0,0,0,0
3,8,22,8,8,8,8,29,8,3,5,...,0,0,0,0,0,0,0,0,0,0
4,8,22,8,8,8,8,29,8,3,5,...,0,0,0,0,0,0,0,0,0,0


In [35]:
preds = rental_cnn.predict(test.values, batch_size = 2*CFG.BATCH_SIZE)
tst = pd.read_parquet(f'/content/drive/MyDrive/BELKA_model/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv(f'/content/drive/MyDrive/BELKA_model/submission_{name}.csv', index = False)

205/205 [==============================] - 15s 71ms/step


In [ ]:
# def make_val_prediction(model, name, logger, lib='tf'):
#     if lib=='tf':
#         tf_valid_pipe = belka_pipeline(
#         batch_size=CFG.BATCH_SIZE,
#         num_threads=4,
#         device_id=CFG.DEVICE_ID,
#         device=CFG.DEVICE,
#         paths=CFG.VALIDS,
#         idxs=VALID_IDX,
#         seed=CFG.SEED-4
#         )
#         val_loader = DALIDataset(pipeline=tf_valid_pipe,
#                        output_dtypes=(tf.int64, tf.float32),
#                        output_shapes=((CFG.BATCH_SIZE, 142), (CFG.BATCH_SIZE, 3)),
#                        batch_size=CFG.BATCH_SIZE,
#                        device_id=None)
#         preds = model.predict(val_loader)
#     elif lib=='torch':
#         assert torch.cuda.is_available()
#         class LightningWrapper(DALIGenericIterator):
#             def __init__(self, *kargs, **kwargs):
#                 super().__init__(*kargs, **kwargs)
#             def __next__(self):
#                 out = super().__next__()
#                 out = out[0]
#                 return [out[k] for k in self.output_map]

#         torch_valid_pipe = belka_pipeline(
#             batch_size=CFG.BATCH_SIZE,
#             num_threads=4,
#             device_id=CFG.DEVICE_ID,
#             device=CFG.DEVICE,
#             paths=CFG.VALIDS,
#             idxs=VALID_IDX,
#             seed=CFG.SEED-2
#         )
#         val_loader = LightningWrapper(torch_valid_pipe, ['X', 'y'],reader_name='Reader', last_batch_policy=LastBatchPolicy.PARTIAL)
#         metrics_ap = mAP(3,thresholds=None, average='none')
#         metrics_map = mAP(3, thresholds=None, average='micro')
#         all_preds = []
#         all_y = []
#         model.to(0)
#         model.eval()
#         with torch.no_grad():
#             for X, y in val_loader:
#                 oof = model(X)
#                 all_preds.append(oof)
#                 all_y.append(y)
#         preds = torch.cat(all_preds, 0)
#         y_eval = torch.cat(all_y, 0).to(torch.int)

#         APs = metrics_ap(preds, y_eval)
#         for i in range(3):
#             logger.info(f'val_AP_bind{i} : {APs[i]}')
#         meanAP = metrics_map(preds, y_eval)
#         logger.info(f'valid_results: CV score = {meanAP}')
#         preds = preds.clone().to('cpu').detach().numpy()

#     val_results = pd.DataFrame({f'bind{i}': preds[:, i] for i in range(3)})
#     val_results.to_csv(os.path.join(CFG.BASE_DIR, f'val_results_{name}.csv'))
#     logger.info(f'val_results write complite!\nfile_name: val_results_{name}.csv')

In [ ]:
import time

name='SeX'

now = time.localtime()
now = time.strftime("%Y-%m-%d-%H-%M-%S", now)
log_name = f'{name}-{now}.log'
logger = get_my_logger(CFG.BASE_DIR, log_name)

In [ ]:

SeX_model.load_weights('/content/drive/MyDrive/BELKA_model/model-SeXception_v2.h5')
SeXc_v3_model.load_weights('/content/drive/MyDrive/BELKA_model/model-SeXception_v3.h5')
make_val_prediction(SeX_model, 'SeX', logger)